In this tutorial, we cover the `pyphs.Simulation` object for the numerical simulation of `pyphs.Core` objects. The core object from the previous tutorial on [`pyphs.Core`](https://afalaize.github.io/pyphs/posts/pyphs-core/) is used as an example.

<!-- TEASER_END -->

The corresponding Python script `pyphs-simulation.py` can be found in the tutorials at `\pyphs\tutorials\`

In [ ]:
# Plot appears in the notebook
%matplotlib inline

In [ ]:
# Support for Python 2.x and Python 3.x
from __future__ import division

# Disable warnings 
import warnings; 
warnings.simplefilter('ignore')

# import of external packages
import numpy                     # numerical tools
import matplotlib.pyplot as plt  # plot tools

In [3]:
from pyphs.tutorials.core import core 
from pyphs import Simulation

Remove Inverse of Parameters...


In [7]:
# Define the simulation parameters
config = {'fs': 48e3,               # Sample rate (Hz)
          'grad': 'discret',    # gradient evaluation in {'discret', 'theta', 'trapez'}
          'theta': 0.5,             # theta for evaluation of the structure
          'split': True,            # Split and presolve the explicit equations
          'maxit': 10,              # Max number of iterations for NL solvers
          'eps': 1e-11,          # Global numerical tolerance
          'path': None,             # Path to saved results. If None, /data/ is created.
          'pbar': False,     # Display a progress bar
          'timer': False            # Display minimal timing infos
          }

# Instantiate a pyphs.Simulation object associated with a given core PHS

simu = core.to_simulation(config = config)

Build method my_core...
Build numeric my_core...
Build data i/o...


In [8]:
# def simulation time
tmax = 0.02
nmax = int(tmax*simu.config['fs'])
t = [n/simu.config['fs'] for n in range(nmax)]


# def input signal
def sig(t, mode='sin'):
    F = 1000.
    A = 1000.
    if mode == 'sin':
        out = A*numpy.sin(2*numpy.pi*F*t)
    elif mode == 'impact':
        dur = 0.5*1e-3
        start = 0.001
        out = A if start <= t < start + dur else 0.
    elif mode == 'none':
        out = 0.
    return numpy.array([out])


# def generator for sequence of inputs to feed in the PHSSimulation object
def sequ(mode='impact'):
    for tn in t:
        yield sig(tn, mode)

In [9]:
# Initialize the simulation
simu.init(u=sequ(), nt=len(t))

# Proceed
simu.process()

Simulation: Process...
Simulation: Done


In [10]:
simu.data.x(1)

<generator object data_generator at 0x7f8a2a17bba0>

In [11]:
# The simulation results are stored in the simu.data object
t = simu.data.t()       # a generator of time value at each time step
x = simu.data.x()       # a generator of value for vector x at each time step
x1 = simu.data.x(1)    # a generator of value for scalar x component 1

t_list = list(t)
x_list = list(x)
x1_list = list(x1)

plt.figure()
plt.plot(t_list, x1_list)
plt.xlabel('$t \; \mathrm{(s)}$')
plt.ylabel('$x_L \; \mathrm{(Wb)}$')

In [12]:
plt.figure()
plt.plot([ex[0] for ex in x_list], [ex[1] for ex in x_list])
plt.xlabel('$x_L\; \mathrm{(Wb)}$')
plt.ylabel('$x_C \; \mathrm{(C)}$')

In [ ]:
plt.figure()
simu.data.plot([('u', 0), ('x', 0), ('dtx', 0), ('y', 0)])

In [ ]:
simu.data.plot_powerbal(mode='single')